In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flask flask-cors joblib pandas numpy requests
!pip install pyngrok

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
import torch
import joblib
import numpy as np
import pandas as pd
import requests
import os
import json
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import threading
import time
import signal
import sys
from datetime import datetime

# IMPORTANT: Update this URL in your Django views.py file!
# Replace 'https://your-ngrok-url.ngrok.io' with the actual URL shown below

In [9]:
%cd /content/drive/MyDrive/crypto-predicter-main

/content/drive/MyDrive/crypto-predicter-main


In [10]:
# !git clone https://github.com/duongphamminhdung/crypto-predicter
# %cd crypto-predicter
# !git checkout -b model_attention_1
# !git checkout -b furthur_debug
# !git reset --hard HEAD
# !git pull

In [11]:
%ls

data_processor.py  mexc_client.py   README.md          train.py
debug_labels.py    model.py         requirements.txt   web_dashboard/
debug_nans.py      predict_live.py  run.sh
exchange.py        __pycache__/     training_data.csv


In [12]:
!sh run.sh install


═══════════════════════════════════════════════════════════
  Installing Dependencies
═══════════════════════════════════════════════════════════

/usr/bin/python3
✅ Python detected: Python 3.12.12
ℹ️  Installing packages from requirements.txt...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!sh run.sh train


═══════════════════════════════════════════════════════════
  Training Model
═══════════════════════════════════════════════════════════

/usr/bin/python3
✅ Python detected: Python 3.12.12
ℹ️  Starting initial model training...
ℹ️  This will:
ℹ️    • Fetch ~100,000 minutes (~2 months) of BTC/USDT Min1 data
ℹ️    • Calculate 90+ technical indicators & features
ℹ️    • Train a 4-layer Transformer model for 20 epochs
ℹ️    • Save weights to ../model/btc_predicter_model.pth

📁 Model directory: /content/drive/MyDrive/model
🏗️  Training parameters
   • look_back      : 60 minutes
   • future_horizon : 240 minutes (4h)
   • tp_sl_ratio    : 0.5% move threshold
Fetching initial training data
MEXC Futures API returns up to 2000 records per request, fetching multiple batches...
Target: 100000 records (~2 months of 1-minute data)
Fetching batch 1/50...
✅ Fetched 2000 kline records for BTC_USDT (Min1)
Fetching batch 2/50...
✅ Fetched 2001 kline records for BTC_USDT (Min1)
Fetching batch 3/50...
✅

In [13]:
!sh run.sh start


═══════════════════════════════════════════════════════════
  Starting Live Trading Bot
═══════════════════════════════════════════════════════════

/usr/bin/python3
✅ Python detected: Python 3.12.12
⚠️  TEST MODE is ENABLED - No real money will be used
ℹ️  To enable live trading, edit predict_live.py and set: TEST = False
ℹ️  Starting the bot...
ℹ️  The bot will:
ℹ️    • Make predictions every 60 seconds
ℹ️    • Use 60-minute input window with 22 technical indicators
ℹ️    • Execute trades when confidence ≥ 0.65
ℹ️    • Auto fine-tune with time-weighted learning when confidence < 0.4 or trade loss
ℹ️    • Log all activity to trading_bot.log
ℹ️    • Save daily stats to trading_stats.json

ℹ️  Press Ctrl+C to stop the bot

2025-12-28 07:33:59,134 [INFO] 🚀 Starting trading bot...
🧠 Transformer model configured to use device: cpu
🧠 Model input size: 92 features (technical indicators)
🧠 Transformer config: d_model=128, nhead=8, num_layers=4
🧠 Transformer model configured to use device: cp

In [ ]:
# Global bot state
bot_running = False
bot_thread = None

app = Flask(__name__)
CORS(app)

def bot_main_loop():
    """Main bot trading loop - integrate with your predict_live.py logic"""
    global bot_running
    print("🤖 Bot started!")

    while bot_running:
        try:
            print("🔄 Bot cycle running...")
            !sh run.sh start
            time.sleep(60)

        except Exception as e:
            print(f"❌ Bot error: {e}")
            time.sleep(5)

    print("🛑 Bot stopped!")

def start_bot():
    """Start the bot in a separate thread"""
    global bot_running, bot_thread

    if bot_running:
        return {"success": False, "message": "Bot is already running"}

    bot_running = True
    bot_thread = threading.Thread(target=bot_main_loop, daemon=True)
    bot_thread.start()

    return {"success": True, "message": "Bot started successfully", "status": "running"}

def stop_bot():
    """Stop the bot"""
    global bot_running, bot_thread

    if not bot_running:
        return {"success": False, "message": "Bot is not running"}

    bot_running = False

    if bot_thread and bot_thread.is_alive():
        bot_thread.join(timeout=10)

    return {"success": True, "message": "Bot stopped successfully", "status": "stopped"}

@app.route('/bot_status', methods=['GET'])
def get_bot_status():
    """Get current bot status"""
    status = "running" if bot_running else "stopped"
    return jsonify({
        "success": True,
        "status": status,
        "timestamp": time.time()
    })

@app.route('/toggle_bot', methods=['POST'])
def toggle_bot():
    """Toggle bot running state"""
    try:
        data = request.get_json() or {}
        action = data.get('action', 'toggle')

        if action == 'start':
            result = start_bot()
        elif action == 'stop':
            result = stop_bot()
        elif action == 'toggle':
            if bot_running:
                result = stop_bot()
            else:
                result = start_bot()
        else:
            result = {"success": False, "message": "Invalid action"}

        return jsonify(result)

    except Exception as e:
        return jsonify({
            "success": False,
            "message": f"Error: {str(e)}"
        })

@app.route('/trading_data', methods=['GET'])
def get_trading_data():
    """Get trading data (trades, stats, etc.)"""
    try:
        # Load trading data from your JSON files
        base_path = '/content/drive/MyDrive/crypto-predicter-main'

        # Try to load from current_trades.json
        current_trades_path = os.path.join(base_path, 'current_trades.json')
        active_trades = []
        if os.path.exists(current_trades_path):
            try:
                with open(current_trades_path, 'r') as f:
                    content = f.read().strip()
                    if content and content != 'w':
                        data = json.loads(content)
                        active_trades = data.get('active_trades', [])
            except Exception as e:
                print(f"Error reading current_trades.json: {e}")

        # Try to load from trades_log.json
        trades_log_path = os.path.join(base_path, 'trades_log.json')
        trades_history = []
        if os.path.exists(trades_log_path):
            try:
                with open(trades_log_path, 'r') as f:
                    content = f.read().strip()
                    if content and content != 'w':
                        raw_history = json.loads(content)
                        # Clean and format the data
                        for t in raw_history:
                            trades_history.append({
                                'time_stamp': t.get('time_stamp'),
                                'entry_price': t.get('entry price'),
                                'profit_loss_result': t.get('Profit/loss'),
                                'pl_percentage': t.get('PL percentage'),
                                'pl_in_dollar': t.get('PL in $'),
                                'side': 'UNK'
                            })
                        trades_history.reverse()  # Newest first
            except Exception as e:
                print(f"Error reading trades_log.json: {e}")

        # Try to load from trading_stats.json
        stats_path = os.path.join(base_path, 'trading_stats.json')
        daily_stats = {}
        if os.path.exists(stats_path):
            try:
                with open(stats_path, 'r') as f:
                    content = f.read().strip()
                    if content:
                        daily_stats = json.loads(content)
            except Exception as e:
                print(f"Error reading trading_stats.json: {e}")

        return jsonify({
            'success': True,
            'trading_data': {
                'trades_history': trades_history,
                'active_trades': active_trades,
                'daily_stats': daily_stats
            }
        })
    except Exception as e:
        print(f"Error loading trading data: {e}")
        return jsonify({
            'success': False,
            'error': f'Failed to load trading data: {str(e)}'
        })

@app.route('/update_trading_params', methods=['POST'])
def update_trading_params():
    """Update trading parameters in predict_live.py"""
    try:
        data = request.get_json()
        if not data:
            return jsonify({
                'success': False,
                'message': 'No data provided'
            })

        # Extract parameters
        confidence_trade = data.get('confidence_trade')
        confidence_test = data.get('confidence_test')
        max_risk = data.get('max_risk')
        max_leverage = data.get('max_leverage')
        max_time_red = data.get('max_time_red')
        opposite_signal = data.get('opposite_signal')
        retrain_interval = data.get('retrain_interval')

        # Validate parameters
        if not all([confidence_trade is not None, confidence_test is not None,
                   max_risk is not None, max_leverage is not None,
                   max_time_red is not None, opposite_signal is not None,
                   retrain_interval is not None]):
            return jsonify({
                'success': False,
                'message': 'Missing required parameters'
            })

        # Validate ranges
        if not (0 <= confidence_trade <= 1 and 0 <= confidence_test <= 1 and
                0 <= max_risk <= 1 and 1 <= max_leverage <= 125 and
                1 <= max_time_red <= 1440 and isinstance(opposite_signal, bool) and
                60 <= retrain_interval <= 86400):
            return jsonify({
                'success': False,
                'message': 'Invalid parameter values'
            })

        # Path to predict_live.py on Drive
        predict_live_path = '/content/drive/MyDrive/crypto-predicter-main/predict_live.py'

        if not os.path.exists(predict_live_path):
            return jsonify({
                'success': False,
                'message': 'predict_live.py not found on Drive'
            })

        # Read the file
        with open(predict_live_path, 'r') as f:
            lines = f.readlines()

        # Update lines 88-97 (0-indexed as 87-96)
        # CONFIDENCE_THRESHOLD_TRADE = 0.70
        # CONFIDENCE_THRESHOLD_TEST = 0.70
        # MAX_POSITION_RISK = 0.10
        # MAX_LEVERAGE = 75
        # EARLY_STOP_MAX_TIME_MINUTES = 120
        # EARLY_STOP_OPPOSITE_SIGNAL = True
        # REFINEMENT_INTERVAL_SECONDS = 3600

        lines[87] = f"CONFIDENCE_THRESHOLD_TRADE = {confidence_trade}\n"
        lines[88] = f"CONFIDENCE_THRESHOLD_TEST = {confidence_test}\n"
        lines[89] = f"MAX_POSITION_RISK = {max_risk}\n"
        lines[90] = f"MAX_LEVERAGE = {max_leverage}\n"
        lines[92] = f"EARLY_STOP_MAX_TIME_MINUTES = {max_time_red}\n"
        lines[93] = f"EARLY_STOP_OPPOSITE_SIGNAL = {opposite_signal}\n"
        lines[96] = f"REFINEMENT_INTERVAL_SECONDS = {retrain_interval}\n"

        # Write back to file
        with open(predict_live_path, 'w') as f:
            f.writelines(lines)

        print(f"✅ Updated trading parameters: TRADE={confidence_trade}, TEST={confidence_test}, RISK={max_risk}, LEVERAGE={max_leverage}, TIME_RED={max_time_red}, OPPOSITE={opposite_signal}, RETRAIN={retrain_interval}")

        return jsonify({
            'success': True,
            'message': 'Trading parameters updated successfully',
            'updated_params': {
                'confidence_trade': confidence_trade,
                'confidence_test': confidence_test,
                'max_risk': max_risk,
                'max_leverage': max_leverage,
                'max_time_red': max_time_red,
                'opposite_signal': opposite_signal,
                'retrain_interval': retrain_interval
            }
        })

    except Exception as e:
        print(f"❌ Error updating trading params: {e}")
        return jsonify({
            'success': False,
            'message': f'Error updating parameters: {str(e)}'
        })

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint"""
    return jsonify({
        'success': True,
        'status': 'healthy',
        'timestamp': time.time()
    })

def signal_handler(signum, frame):
    """Handle shutdown signals"""
    print("\n🛑 Shutting down bot control API...")
    stop_bot()
    sys.exit(0)

def run_api_server():
    """Run the Flask API server"""
    try:
        # Note: Signal handling removed because it only works in main thread
        # The server will be managed by the thread lifecycle instead
        
        print("🚀 Starting Colab Bot Control API...")
        print("📡 API endpoints:")
        print("   GET  /bot_status         - Get bot status")
        print("   POST /toggle_bot         - Toggle bot (start/stop)")
        print("   GET  /trading_data       - Get trading data")
        print("   GET  /predict            - Get model prediction")
        print("   POST /update_trading_params - Update trading parameters")
        print("   GET  /health             - Health check")
        print()

        # Run Flask app
        print("🔧 Starting Flask app on port 8000...")
        app.run(host='0.0.0.0', port=8000, debug=False, use_reloader=False)
        
    except Exception as e:
        print(f"❌ Failed to start Flask server: {e}")
        print("🔍 This might be due to:")
        print("   - Port 8000 already in use")
        print("   - Import errors in the Flask routes")
        print("   - Missing dependencies")
        raise

# Test Flask app initialization before starting server
print("🔍 Testing Flask app initialization...")
try:
    with app.test_client() as client:
        response = client.get('/health')
        if response.status_code == 200:
            print("✅ Flask app initialization test passed!")
        else:
            print(f"⚠️  Flask app test returned status {response.status_code}")
            print(f"Response: {response.get_data(as_text=True)}")
except Exception as e:
    print(f"❌ Flask app initialization test failed: {e}")
    print("🔍 This might indicate issues with:")
    print("   - Route definitions")
    print("   - Import errors")
    print("   - Flask app configuration")
    
    # Try to identify the specific issue
    try:
        import traceback
        traceback.print_exc()
    except:
        pass
    
    print("\n🔧 Attempting to identify the issue...")
    # Test individual imports
    try:
        from flask import Flask, request, jsonify
        print("✅ Flask imports OK")
    except Exception as e:
        print(f"❌ Flask import error: {e}")
    
    try:
        from flask_cors import CORS
        print("✅ Flask-CORS import OK")
    except Exception as e:
        print(f"❌ Flask-CORS import error: {e}")
    
    raise Exception("Flask app initialization failed - check imports and route definitions above")

# Start Flask API in background thread
print("🔧 Starting Flask server thread...")
api_thread = threading.Thread(target=run_api_server, daemon=True)
api_thread.start()

# Wait for Flask server to start up
print("⏳ Waiting for Flask server to start...")
max_attempts = 10
server_started = False

for attempt in range(max_attempts):
    time.sleep(1)
    if check_server(8000):
        print(f"✅ Flask server is running on port 8000 (attempt {attempt + 1})")
        server_started = True
        break
    else:
        print(f"⏳ Server not ready yet (attempt {attempt + 1}/{max_attempts})")
        if attempt == max_attempts - 1:
            print("❌ Flask server failed to start after maximum attempts")
            print("🔍 Checking thread status...")
            print(f"   Thread alive: {api_thread.is_alive()}")
            print(f"   Thread daemon: {api_thread.daemon}")
            
            # Try alternative ports
            print("🔄 Trying alternative ports...")
            for alt_port in [8000, 8080, 3000, 4000]:
                if not check_server(alt_port):
                    print(f"✅ Port {alt_port} is available, but server needs restart")
                    print(f"💡 Try changing the port in run_api_server() to {alt_port}")
                    break
            raise Exception("Flask server failed to start on port 8000")

# Additional verification
print("🔍 Verifying server with health check...")
try:
    response = requests.get('http://localhost:8000/health', timeout=5)
    if response.status_code == 200:
        print("✅ Health check passed!")
    else:
        print(f"⚠️  Health check returned status {response.status_code}")
except Exception as e:
    print(f"⚠️  Health check failed: {e}")
    print("🔄 Continuing anyway...")

# Set up ngrok tunnel
print("🌐 Setting up ngrok tunnel...")
try:
    ngrok.set_auth_token('37783xOKZI9CZpgim6WRLSVLmH3_64ymLMWnwTuxhuSsPUeEo')  # Get from https://dashboard.ngrok.com/
    public_url = ngrok.connect(8000)
    print(f"🚀 API is running at: {public_url}")
    print(f"🤖 Bot control: {public_url}/toggle_bot")
    print(f"📊 Status: {public_url}/bot_status")
    print(f"📡 Prediction: {public_url}/predict")
    print(f"📊 Trading data: {public_url}/trading_data")
    print(f"⚙️  Update params: {public_url}/update_trading_params")
    print(f"💓 Health: {public_url}/health")
    
    # Store the URL for Django
    colab_api_base_url = str(public_url)
    print(f"\n📝 Update this URL in your Django views.py:")
    print(f"   colab_api_url = '{colab_api_base_url}'")
    
except Exception as e:
    print(f"❌ Failed to setup ngrok tunnel: {e}")
    print("🔍 Possible causes:")
    print("   - Invalid ngrok auth token")
    print("   - ngrok service issues")
    print("   - Port already in use by another ngrok tunnel")
    print("💡 Try: !pkill ngrok  # to kill existing tunnels")
    raise

# Keep the notebook running
print("\n🔄 API server started. Notebook will keep running...")
while True:
    time.sleep(60)
    status = "running" if bot_running else "stopped"
    print(f"🔄 API still running at {datetime.now()} | Bot: {status}")